In [ ]:
import torch
import torchaudio
import matplotlib.pyplot as plt
import librosa
import os
from audio_segment import audio_segment

### Music Dataset

In [ ]:
music_data_raw_folder = r"_Data/Music/raw"
save_folder = r"_Data/Music"
SR = 8000

In [ ]:
data_set = []
for label in os.listdir(music_data_raw_folder):
    curr_size = len(data_set)
    print("Processing label: ", label)
    label_folder_path = os.path.join(music_data_raw_folder, label)
    for audio_file in os.listdir(label_folder_path):
        audio_file_path = os.path.join(label_folder_path, audio_file)
        waveform, orig_sample_rate = librosa.load(audio_file_path)
        waveform = torch.tensor(waveform)
        if orig_sample_rate != SR:
            if orig_sample_rate < SR:
                print(
                    f"Sample rate of {orig_sample_rate} is less than target sample rate {SR}. Skipping {audio_file_path}"
                )
                raise
            resampler = torchaudio.transforms.Resample(
                orig_freq=orig_sample_rate, new_freq=SR
            )
            waveform = resampler(waveform)
        segment_length = 10 * SR
        batches = waveform.split(segment_length)
        for batch_seg in batches:
            if len(batch_seg) < segment_length:
                continue
            data_set.append(audio_segment(data=batch_seg, label=label, org_file=audio_file_path))
    print(f"Added {len(data_set) - curr_size} segments for label {label}")



Processing label:  Angry
Added 120 segments for label Angry
Processing label:  Joy
Added 215 segments for label Joy
Processing label:  Sad
Added 154 segments for label Sad
Processing label:  Scary
Added 161 segments for label Scary
